In [ ]:

# Import dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load the data
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Display the first few rows
application_df.head()


In [ ]:

# Drop the non-beneficial ID columns
application_df = application_df.drop(columns=["EIN", "NAME"])

# Grouping rare categories in 'APPLICATION_TYPE' and 'CLASSIFICATION'
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = application_type_counts[application_type_counts < 1000].index
for app in application_types_to_replace:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app, "Other")

classification_counts = application_df["CLASSIFICATION"].value_counts()
classifications_to_replace = classification_counts[classification_counts < 500].index
for cls in classifications_to_replace:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")

# Convert categorical data to numeric using pd.get_dummies
application_df = pd.get_dummies(application_df)

# Define features and target
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:

# Attempt 1: Increase the number of neurons in hidden layers

# Define the model
nn = tf.keras.models.Sequential()

# Add input and hidden layers
nn.add(tf.keras.layers.Dense(units=256, input_dim=X_train_scaled.shape[1], activation="relu"))
nn.add(tf.keras.layers.Dense(units=128, activation="relu"))

# Add output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Attempt 1 - Loss: {model_loss}, Accuracy: {model_accuracy}")


In [ ]:

# Attempt 2: Add more layers and try different activation functions

# Define the model
nn = tf.keras.models.Sequential()

# Add input and hidden layers with different activation functions
nn.add(tf.keras.layers.Dense(units=256, input_dim=X_train_scaled.shape[1], activation="relu"))
nn.add(tf.keras.layers.Dense(units=128, activation="tanh"))
nn.add(tf.keras.layers.Dense(units=64, activation="relu"))

# Add output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Attempt 2 - Loss: {model_loss}, Accuracy: {model_accuracy}")


In [ ]:

# Attempt 3: Increase the number of epochs and adjust batch size

# Define the model
nn = tf.keras.models.Sequential()

# Add input and hidden layers
nn.add(tf.keras.layers.Dense(units=128, input_dim=X_train_scaled.shape[1], activation="relu"))
nn.add(tf.keras.layers.Dense(units=64, activation="relu"))

# Add output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model with more epochs and adjusted batch size
history = nn.fit(X_train_scaled, y_train, epochs=200, batch_size=64, validation_data=(X_test_scaled, y_test))

# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Attempt 3 - Loss: {model_loss}, Accuracy: {model_accuracy}")


In [ ]:

# Save the final model to an HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")
